<a href="https://colab.research.google.com/github/jaeyeon1234/hondl/blob/main/hondeeplearning02_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#입력 준비 및 정규화 적용하기
import keras
from keras import layers
inputs = layers.Input(shape=(224,224,3))
x=layers.ZeroPadding2D(padding=1)(inputs)  #이미지 주변에 3픽셀의 패딩 추가..
x=layers.Conv2D(64,7,strides=2)(x)
x=layers.BatchNormalization(epsilon=1e-5)(x)
x=layers.Activation('relu')(x)


평균과 분산을을 계산하여 기록하고 이걸로 예측에 사용해 배치 정규화 수행

In [2]:
#패딩 추가 및 최대 풀링 적용하기
x = layers.ZeroPadding2D(padding=1)(x)
x = layers.MaxPooling2D(pool_size=3, strides=2)(x)

In [3]:
#잔차 스택 만들기
def build_stack(x):
  for blocks, filters in [(3,64),(4,128), (6,256),(3,512)]:
    x = residual_stack(x, blocks, filters)
  return x

def residual_stack(x, blocks, filters):
  for _ in range(blocks):
    x = residual_block(x, filters)
  return x


In [5]:
#잔차 스택 수정
def build_stack(x):
  #첫번째 잔차 스택의 첫번째 잔차 블록만 스트라이드 1 사용
  x=residual_stack(x, 3, 64, first_stride=1)
  #2~4번째 잔차 블록 만듦
  for blocks, filters in [(4,128), (6,256),(3,512)]:
    x = residual_stack(x, blocks, filters)
  return x

def residual_stack(x, blocks, filters, first_stride=2):
  #첫번째 잔차 블록의 첫번째 합성곱 스트라이드는 first_stride임
  x = residual_block(x, filters, first_stride=first_stride)
  for _ in range(1, blocks):
    #나머지 블록은 스트라이드 1
    x = residual_block(x, filters, first_stride=1)
  return x

In [7]:
#또 수정.. 좀 한번에 하자;;
def build_stack(x):
  #첫번째 잔차 스택의 첫번째 잔차 블록만 스트라이드 1 사용
  x=residual_stack(x, 3, 64, first_stride=1)
  #2~4번째 잔차 블록 만듦
  for blocks, filters in [(4,128), (6,256),(3,512)]:
    x = residual_stack(x, blocks, filters, first_stride=2)
  return x

def residual_stack(x, blocks, filters, first_stride=2):
  #첫 번째 잔차 블록은 합성곱 스킵 연결 사용
  #첫번째 잔차 블록의 첫번째 합성곱 스트라이드는 first_stride임
  x = residual_block(x, filters, first_stride=first_stride, conv_skip=True)
  for _ in range(1, blocks):
    #나머지 블록은 스트라이드 1
    x = residual_block(x, filters, first_stride=1, conv_skip=False)
  return x

In [8]:
#잔차 블록 만들기
def residual_block(x, filters, first_stride=1, conv_skip=False):
  skip_conn = x
  #합성곱과 배치 정규화, 렐루 활성화
  # 1*1 , filters개 필터, 스트라이드는 first_stride에 따라 1 또는 2
  x=layers.Conv2D(filters=filters, kernel_size=1,
                  strides=first_stride)(x)
  x = layers.BatchNormalization(epsilon=1e-5)(x)
  x = layers.Activation('relu')(x)
 #3*3, filters개 필터
  x = layers.Conv2D(filters=filters, kernel_size=3,
                    strides=1, padding='same')(x)
  x=layers.BatchNormalization(epsilon=1e-5)(x)
  x=layers.Activation('relu')(x)
  #1*1, filters*4개 필터
  x=layers.Conv2D(filters=filters*4, kernel_size=1)(x)
  x=layers.BatchNormalization(epsilon=1e-5)(x)

  #conv_skip이 True 이면 1*1 합성곱을 사용해 채널 크기를 filters*4개로 늘림
  if conv_skip==True:
    skip_conn = layers.Conv2D(filters=filters*4, kernel_size=1,
                              strides=first_stride)(skip_conn)
    skip_conn = layers.BatchNormalization(epsilon=1e-5)(skip_conn)
  x=layers.Add()([skip_conn, x])
  x=layers.Activation('relu')(x)
  return x

In [9]:
#ResNet 모델 만들기
x= build_stack(x)

In [10]:
x=layers.GlobalAveragePooling2D()(x)
outputs = layers.Dense(1000, activation='softmax')(x)
model = keras.Model(inputs, outputs)

In [11]:
#ResNet101 모델 만들기 (101개 층으로 구성됨)
def build_stack101(x):
  x=residual_stack(x,3,64, first_stride=1)

  for blocks, filters in[(4,128),(23,256),(3,512)]:
    x= residual_stack(x, blocks, filters, first_stride=2)
  return x

In [12]:
#ResNet152 모델 만들기(152개 층으로 구성됨)
def build_stack152(x):
  x = residual_stack(x, 3, 64, first_stride=1)
  for blocks, filters in [(8,128),(36,256),(3,512)]:
    x = residual_stack(x, blocks, filters, first_stride=2)
  return x

In [14]:
#강아지와 고양이 사진 분류하기
!gdown 1xGkTT3uwYt4myj6eJJeYtdEFgTi2Sj8C
!unzip cat-dog-images.zip

Downloading...
From: https://drive.google.com/uc?id=1xGkTT3uwYt4myj6eJJeYtdEFgTi2Sj8C
To: /content/cat-dog-images.zip
100% 182k/182k [00:00<00:00, 69.6MB/s]
Archive:  cat-dog-images.zip
   creating: images/
  inflating: images/dog.png          
  inflating: images/cat.png          


In [15]:
#ResNet50 모델 사용하여 강아지 사진 예측
from PIL import Image
import numpy as np
from keras.applications import resnet

dog_png = Image.open('images/dog.png')
resnet_prep_dog = resnet.preprocess_input(np.array(dog_png))

In [16]:
resnet50 = keras.applications.ResNet50()
predictions = resnet50.predict(resnet_prep_dog[np.newaxis,:])

resnet.decode_predictions(predictions)

102967424/102967424 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
35363/35363 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


[[('n02099712', 'Labrador_retriever', np.float32(0.3853521)),
  ('n02099601', 'golden_retriever', np.float32(0.089699686)),
  ('n02100735', 'English_setter', np.float32(0.04212423)),
  ('n02106166', 'Border_collie', np.float32(0.037774343)),
  ('n02101388', 'Brittany_spaniel', np.float32(0.030700468))]]

VGG16 모델보다 래브라도 리트리버에 대한 확신이 큼(38.5%>35.7%)

In [17]:
#고양이 사진 분류하기
cat_png = Image.open('images/cat.png')
resnet_prep_cat = resnet.preprocess_input(np.array(cat_png))
predictions = resnet50.predict(resnet_prep_cat[np.newaxis,:])
resnet.decode_predictions(predictions)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 244ms/step


[[('n02123045', 'tabby', np.float32(0.86861026)),
  ('n02124075', 'Egyptian_cat', np.float32(0.05077493)),
  ('n02123159', 'tiger_cat', np.float32(0.042566977)),
  ('n07930864', 'cup', np.float32(0.0027631463)),
  ('n03443371', 'goblet', np.float32(0.002099165))]]

VGG16 모델보다 훨씬 높은 수준으로 얼룩고양이라고 예측(86.8%>43.2%)